# Práctica 2 Opcional

## 1. Marcar el punto de giro con el ratón

In [ ]:
import cv2 as cv
import numpy as np

image = cv.imread('images/cats.png')
if image is None:
    raise FileNotFoundError("No se encontró la imagen.")
h, w = image.shape[:2]

cv.namedWindow("Transformaciones")

cx, cy = w//2, h//2
mouse_selected = False

def nothing(x):
    pass


cv.createTrackbar("Tx", "Transformaciones", w//2, w, nothing)
cv.createTrackbar("Ty", "Transformaciones", h//2, h, nothing)
cv.createTrackbar("Angulo", "Transformaciones", 0, 360, nothing)
cv.createTrackbar("Cx", "Transformaciones", w//2, w, nothing)
cv.createTrackbar("Cy", "Transformaciones", h//2, h, nothing)
cv.createTrackbar("EscalaX", "Transformaciones", 100, 200, nothing)
cv.createTrackbar("EscalaY", "Transformaciones", 100, 200, nothing)
cv.createTrackbar("Uniforme", "Transformaciones", 0, 1, nothing)

def mouse(event, x, y, flags, param):
    global cx, cy, mouse_selected
    if event == cv.EVENT_LBUTTONDOWN:
        cx, cy = x, y
        mouse_selected = True

cv.setMouseCallback("Transformaciones", mouse)

print("Usa sliders o clic en la imagen para cambiar el punto de giro.")
print("Pulsa ESC para salir.")

while True:
    tx = cv.getTrackbarPos("Tx", "Transformaciones") - w//2
    ty = cv.getTrackbarPos("Ty", "Transformaciones") - h//2
    angle = cv.getTrackbarPos("Angulo", "Transformaciones")
    sx = cv.getTrackbarPos("EscalaX", "Transformaciones") / 100.0
    sy = cv.getTrackbarPos("EscalaY", "Transformaciones") / 100.0
    uniforme = cv.getTrackbarPos("Uniforme", "Transformaciones")

    if not mouse_selected:
        cx = cv.getTrackbarPos("Cx", "Transformaciones")
        cy = cv.getTrackbarPos("Cy", "Transformaciones")
    else:
        cv.setTrackbarPos("Cx", "Transformaciones", cx)
        cv.setTrackbarPos("Cy", "Transformaciones", cy)
        mouse_selected = False

    T = np.float32([[1, 0, tx],
                    [0, 1, ty]])
    img_T = cv.warpAffine(image, T, (w, h))

    M_translate1 = np.float32([[1, 0, -cx],
                               [0, 1, -cy]])
    M_rotate = cv.getRotationMatrix2D((0, 0), angle, 1.0)
    M_translate2 = np.float32([[1, 0, cx],
                               [0, 1, cy]])

    M1 = np.vstack([M_translate1, [0, 0, 1]])
    M2 = np.vstack([M_rotate, [0, 0, 1]])
    M3 = np.vstack([M_translate2, [0, 0, 1]])
    M_rot = M3 @ M2 @ M1
    img_R = cv.warpAffine(img_T, M_rot[:2], (w, h))

    if uniforme:
        s = sx
        S = np.float32([[s, 0, 0],
                        [0, s, 0]])
    else:
        S = np.float32([[sx, 0, 0],
                        [0, sy, 0]])

    img_final = cv.warpAffine(img_R, S, (w, h))

    img_display = img_final.copy()
    
    cv.circle(img_display, (cx, cy), 5, (0, 0, 255), -1)
    cv.putText(img_display, f"({cx},{cy})", (cx + 10, cy - 10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    tx_point, ty_point = w//2 + tx, h//2 + ty
    cv.circle(img_display, (tx_point, ty_point), 5, (0, 255, 0), -1)
    cv.putText(img_display, f"({tx_point},{ty_point})", (tx_point + 10, ty_point - 10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv.imshow("Transformaciones", img_display)

    if cv.waitKey(1) & 0xFF == 27:
        break

cv.destroyAllWindows()


Usa sliders o clic en la imagen para cambiar el punto de giro.
Pulsa ESC para salir.


## 2. Trasladar la imagen arrastrando con el ratón

In [ ]:
import cv2
import numpy as np

clone = cv2.imread("images/cats.png")
if clone is None:
    raise FileNotFoundError("No se pudo cargar la imagen.")

dragging = False
start_x, start_y = 0, 0
tx, ty = 0, 0

def mouse_drag(event, x, y, flags, param):
    global dragging, start_x, start_y, tx, ty
    if event == cv2.EVENT_LBUTTONDOWN:
        dragging = True
        start_x, start_y = x, y
    elif event == cv2.EVENT_MOUSEMOVE and dragging:
        tx, ty = x - start_x, y - start_y
    elif event == cv2.EVENT_LBUTTONUP:
        dragging = False

cv2.namedWindow("Traslación con ratón")
cv2.setMouseCallback("Traslación con ratón", mouse_drag)

while True:
    M = np.float32([[1, 0, tx], [0, 1, ty]])
    moved = cv2.warpAffine(clone, M, (clone.shape[1], clone.shape[0]))
    cv2.imshow("Traslación con ratón", moved)

    key = cv2.waitKey(20) & 0xFF
    if key == 27:
        cv2.imwrite("output/imagen_trasladada.png", moved)
        print("Imagen guardada como 'output/imagen_trasladada.png'")
        break

cv2.destroyAllWindows()


Imagen guardada como 'output/imagen_trasladada.png'


### Código unido: Marcar el punto de giro y trasladar la imagen arrastrando con el ratón

In [4]:
import cv2 as cv
import numpy as np

image = cv.imread('images/cats.png')
if image is None:
    raise FileNotFoundError("No se encontró la imagen.")
h, w = image.shape[:2]

cv.namedWindow("Transformaciones")


cx, cy = w//2, h//2
mouse_selected = False
dragging_img = False
start_x, start_y = 0, 0
tx_mouse, ty_mouse = 0, 0

def nothing(x):
    pass


cv.createTrackbar("Tx", "Transformaciones", w//2, w, nothing)
cv.createTrackbar("Ty", "Transformaciones", h//2, h, nothing)
cv.createTrackbar("Angulo", "Transformaciones", 0, 360, nothing)
cv.createTrackbar("Cx", "Transformaciones", w//2, w, nothing)
cv.createTrackbar("Cy", "Transformaciones", h//2, h, nothing)
cv.createTrackbar("EscalaX", "Transformaciones", 100, 200, nothing)
cv.createTrackbar("EscalaY", "Transformaciones", 100, 200, nothing)
cv.createTrackbar("Uniforme", "Transformaciones", 0, 1, nothing)


def mouse(event, x, y, flags, param):
    global cx, cy, mouse_selected
    global dragging_img, start_x, start_y, tx_mouse, ty_mouse

    if event == cv.EVENT_LBUTTONDOWN:
        dragging_img = True
        start_x, start_y = x, y
    elif event == cv.EVENT_MOUSEMOVE and dragging_img:
        tx_mouse, ty_mouse = x - start_x, y - start_y
    elif event == cv.EVENT_LBUTTONUP:
        dragging_img = False

    if event == cv.EVENT_RBUTTONDOWN:
        cx, cy = x, y
        mouse_selected = True

cv.setMouseCallback("Transformaciones", mouse)

print("Trackbars o clic derecho para punto de giro, arrastra con izquierdo para mover imagen.")
print("Pulsa ESC para salir.")

while True:
    tx = cv.getTrackbarPos("Tx", "Transformaciones") - w//2
    ty = cv.getTrackbarPos("Ty", "Transformaciones") - h//2
    angle = cv.getTrackbarPos("Angulo", "Transformaciones")
    sx = cv.getTrackbarPos("EscalaX", "Transformaciones") / 100.0
    sy = cv.getTrackbarPos("EscalaY", "Transformaciones") / 100.0
    uniforme = cv.getTrackbarPos("Uniforme", "Transformaciones")

    if not mouse_selected:
        cx = cv.getTrackbarPos("Cx", "Transformaciones")
        cy = cv.getTrackbarPos("Cy", "Transformaciones")
    else:
        cv.setTrackbarPos("Cx", "Transformaciones", cx)
        cv.setTrackbarPos("Cy", "Transformaciones", cy)

    total_tx = tx + tx_mouse
    total_ty = ty + ty_mouse
    T = np.float32([[1, 0, total_tx],
                    [0, 1, total_ty]])
    img_T = cv.warpAffine(image, T, (w, h))

    M_translate1 = np.float32([[1, 0, -cx],
                               [0, 1, -cy]])
    M_rotate = cv.getRotationMatrix2D((0, 0), angle, 1.0)
    M_translate2 = np.float32([[1, 0, cx],
                               [0, 1, cy]])
    M1 = np.vstack([M_translate1, [0,0,1]])
    M2 = np.vstack([M_rotate, [0,0,1]])
    M3 = np.vstack([M_translate2, [0,0,1]])
    M_rot = M3 @ M2 @ M1
    img_R = cv.warpAffine(img_T, M_rot[:2], (w,h))

    if uniforme:
        s = sx
        S = np.float32([[s,0,0],[0,s,0]])
    else:
        S = np.float32([[sx,0,0],[0,sy,0]])
    img_final = cv.warpAffine(img_R, S, (w,h))

    img_display = img_final.copy()
    cv.circle(img_display, (cx, cy), 5, (0,0,255), -1)
    cv.putText(img_display, f"({cx},{cy})", (cx+10, cy-10), cv.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255),2)

    tx_point, ty_point = w//2 + total_tx, h//2 + total_ty
    cv.circle(img_display, (tx_point, ty_point), 5, (0,255,0), -1)
    cv.putText(img_display, f"({tx_point},{ty_point})", (tx_point+10, ty_point-10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    cv.imshow("Transformaciones", img_display)

    if cv.waitKey(1) & 0xFF == 27:
        break

cv.destroyAllWindows()


Trackbars o clic derecho para punto de giro, arrastra con izquierdo para mover imagen.
Pulsa ESC para salir.


## 3. Hacer la parte obligatoria sobre vídeo
###  1a. 
- Desarrollar una aplicación que lleve a cabo transformaciones de la imagen en tiempo real a través de una interfaz basada en trackbars o equivalente
- Hacer traslaciones. Es necesario indicar la magnitud de la traslación en X y en Y.
- Hacer rotaciones. Es necesario indicar el centro de giro y ángulo de giro.
- Hacer escalados uniformes y no uniformes. Es necesario indicar los factores de
escala.


In [7]:
import cv2 as cv
import numpy as np

cap = cv.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("No se pudo abrir la cámara.")

ret, frame = cap.read()
if not ret:
    raise RuntimeError("No se pudo leer frame de la cámara.")

h, w = frame.shape[:2]

cv.namedWindow("Transformaciones", cv.WINDOW_AUTOSIZE)

def nothing(x):
    pass

# Trackbars
cv.createTrackbar("Tx", "Transformaciones", w // 2, w, nothing)
cv.createTrackbar("Ty", "Transformaciones", h // 2, h, nothing)
cv.createTrackbar("Angulo", "Transformaciones", 0, 360, nothing)
cv.createTrackbar("Cx", "Transformaciones", w // 2, w, nothing)
cv.createTrackbar("Cy", "Transformaciones", h // 2, h, nothing)
cv.createTrackbar("EscalaX", "Transformaciones", 100, 200, nothing)
cv.createTrackbar("EscalaY", "Transformaciones", 100, 200, nothing)
cv.createTrackbar("Uniforme", "Transformaciones", 0, 1, nothing)

print("Usa los sliders para transformar el video en tiempo real.")
print("Pulsa ESC para salir.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    tx = cv.getTrackbarPos("Tx", "Transformaciones") - w // 2
    ty = cv.getTrackbarPos("Ty", "Transformaciones") - h // 2
    angle = cv.getTrackbarPos("Angulo", "Transformaciones")
    cx = cv.getTrackbarPos("Cx", "Transformaciones")
    cy = cv.getTrackbarPos("Cy", "Transformaciones")
    sx = cv.getTrackbarPos("EscalaX", "Transformaciones") / 100.0
    sy = cv.getTrackbarPos("EscalaY", "Transformaciones") / 100.0
    uniforme = cv.getTrackbarPos("Uniforme", "Transformaciones")

    T = np.float32([[1, 0, tx],
                    [0, 1, ty]])
    img_T = cv.warpAffine(frame, T, (w, h))

    M_translate1 = np.float32([[1, 0, -cx],
                               [0, 1, -cy]])
    M_rotate = cv.getRotationMatrix2D((0, 0), angle, 1.0)
    M_translate2 = np.float32([[1, 0, cx],
                               [0, 1, cy]])

    M1 = np.vstack([M_translate1, [0, 0, 1]])
    M2 = np.vstack([M_rotate, [0, 0, 1]])
    M3 = np.vstack([M_translate2, [0, 0, 1]])
    M_rot = M3 @ M2 @ M1
    img_R = cv.warpAffine(img_T, M_rot[:2], (w, h))

    if uniforme:
        s = sx
        S = np.float32([[s, 0, 0],
                        [0, s, 0]])
    else:
        S = np.float32([[sx, 0, 0],
                        [0, sy, 0]])
    img_final = cv.warpAffine(img_R, S, (w, h))

    img_display = img_final.copy()

    # Punto de rotación (rojo)
    cv.circle(img_display, (cx, cy), 5, (0, 0, 255), -1)
    cv.putText(img_display, f"({cx},{cy})", (cx + 10, cy - 10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # Punto de traslación aplicado (verde)
    tx_point, ty_point = w // 2 + tx, h // 2 + ty
    cv.circle(img_display, (tx_point, ty_point), 5, (0, 255, 0), -1)
    cv.putText(img_display, f"({tx_point},{ty_point})", (tx_point + 10, ty_point - 10),
               cv.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv.imshow("Transformaciones", img_display)

    if cv.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv.destroyAllWindows()


Usa los sliders para transformar el video en tiempo real.
Pulsa ESC para salir.


### 1b. 
- Dada una imagen trazar una ventana de proyección y proyectar la imagen.

In [9]:
import cv2
import numpy as np

src_points = []
dst_points = []
phase = 0
frame_ref = None
homography_ready = False
H = None
dst_polygon = None

def ordenar_cuatro_puntos(pts):
    pts = np.array(pts, dtype=np.float32)
    s = pts.sum(axis=1)
    diff = np.diff(pts, axis=1)

    tl = pts[np.argmin(s)]
    br = pts[np.argmax(s)]
    tr = pts[np.argmin(diff)]
    bl = pts[np.argmax(diff)]
    return np.array([tl, tr, br, bl], dtype=np.float32)

def mouse_callback(event, x, y, flags, param):
    global src_points, dst_points, phase, frame_ref, H, homography_ready, dst_polygon

    if event == cv2.EVENT_LBUTTONDOWN:
        if phase == 0 and len(src_points) < 4:
            src_points.append([x, y])
            if len(src_points) == 4:
                print("Selecciona ahora los 4 puntos destino (rojo).")
                phase = 1

        elif phase == 1 and len(dst_points) < 4:
            dst_points.append([x, y])
            if len(dst_points) == 4:
                print("Calculando proyección...")
                src = ordenar_cuatro_puntos(src_points)
                dst = ordenar_cuatro_puntos(dst_points)
                H, _ = cv2.findHomography(src, dst)
                dst_polygon = np.int32(dst)
                homography_ready = True

def main():
    global frame_ref, H, homography_ready, dst_polygon

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise RuntimeError("No se pudo abrir el video/cámara.")

    ret, frame_ref = cap.read()
    if not ret:
        raise RuntimeError("No se pudo leer el primer frame.")

    cv2.namedWindow("Proyeccion interactiva")
    cv2.setMouseCallback("Proyeccion interactiva", mouse_callback)

    print("Haz click en 4 puntos fuente (verde), luego en 4 puntos destino (rojo).")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        output = frame.copy()

        if homography_ready and H is not None:
            h, w = frame.shape[:2]
            warped = cv2.warpPerspective(frame, H, (w, h))

            mask = np.zeros((h, w), dtype=np.uint8)
            cv2.fillPoly(mask, [dst_polygon], 255)

            result = np.zeros_like(frame)
            cv2.copyTo(warped, mask, result)

            output = result

        for (x, y) in src_points:
            cv2.circle(output, (int(x), int(y)), 6, (0, 255, 0), -1)
        for (x, y) in dst_points:
            cv2.circle(output, (int(x), int(y)), 6, (0, 0, 255), -1)

        cv2.imshow("Proyeccion interactiva", output)

        key = cv2.waitKey(30) & 0xFF
        if key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Haz click en 4 puntos fuente (verde), luego en 4 puntos destino (rojo).
Selecciona ahora los 4 puntos destino (rojo).
Calculando proyección...


## 4. Transformación afín con 3 puntos

In [12]:
import cv2
import numpy as np

src_points = []
dst_points = []
phase = 0
image = None
clone = None

def mouse_callback(event, x, y, flags, param):
    global src_points, dst_points, phase, image, clone

    if event == cv2.EVENT_LBUTTONDOWN:
        if phase == 0 and len(src_points) < 3:
            src_points.append([x, y])
            cv2.circle(image, (x, y), 5, (0, 255, 0), -1)
            cv2.imshow("Transformacion Afin", image)

            if len(src_points) == 3:
                print("Selecciona ahora los 3 puntos destino (rojo).")
                phase = 1

        elif phase == 1 and len(dst_points) < 3:
            dst_points.append([x, y])
            cv2.circle(image, (x, y), 5, (0, 0, 255), -1)
            cv2.imshow("Transformacion Afin", image)

            if len(dst_points) == 3:
                print("Calculando transformación afín...")
                aplicar_transformacion()

def aplicar_transformacion():
    global src_points, dst_points, clone, image

    src = np.array(src_points, dtype=np.float32)
    dst = np.array(dst_points, dtype=np.float32)

    M = cv2.getAffineTransform(src, dst)

    h, w = clone.shape[:2]
    warped = cv2.warpAffine(clone, M, (w, h))

    cv2.imshow("Transformacion Afin", warped)
    cv2.imwrite("output/resultado_affine.png", warped)
    print("Resultado guardado en 'resultado_affine.png'")

def main():
    global image, clone

    ruta = "images/cats.png"
    clone = cv2.imread(ruta)
    if clone is None:
        raise FileNotFoundError("No se pudo cargar la imagen.")
    image = clone.copy()

    cv2.namedWindow("Transformacion Afin")
    cv2.setMouseCallback("Transformacion Afin", mouse_callback)
    cv2.imshow("Transformacion Afin", image)

    print("Haz click en 3 puntos fuente (verde), luego en 3 puntos destino (rojo).")
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Haz click en 3 puntos fuente (verde), luego en 3 puntos destino (rojo).
Selecciona ahora los 3 puntos destino (rojo).
Calculando transformación afín...
Resultado guardado en 'resultado_affine.png'


## 5. Calcular la imagen especular a partir de una imagen.

In [ ]:
import cv2

ruta = "images/cats.png"
img = cv2.imread(ruta)
if img is None:
    raise FileNotFoundError("No se pudo cargar la imagen.")

espejo_horizontal = cv2.flip(img, 1)
espejo_vertical = cv2.flip(img, 0)
espejo_completo = cv2.flip(img, -1)

cv2.imshow("Original", img)
cv2.imshow("Espejo horizontal", espejo_horizontal)
cv2.imshow("Espejo vertical", espejo_vertical)
cv2.imshow("Espejo completo", espejo_completo)

cv2.imwrite("output/espejo_horizontal.png", espejo_horizontal)
cv2.imwrite("output/espejo_vertical.png", espejo_vertical)
cv2.imwrite("output/espejo_completo.png", espejo_completo)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 6. Trazar una recta que será el eje de reflexión y “reflejar” la imagen.

In [21]:
import cv2
import numpy as np
import os

points = []
clone = None
done = False

os.makedirs("output", exist_ok=True)

def mouse_callback(event, x, y, flags, param):
    global points, clone, done

    if done:
        return

    if event == cv2.EVENT_LBUTTONDOWN:
        points.append([x, y])
        if len(points) == 2:
            reflected_clean = warp_reflection(clone, points[0], points[1])
            cv2.imwrite("output/resultado_reflexion.png", reflected_clean)

            reflected_with_line = actualizar_reflexion(points[1])
            cv2.imwrite("output/resultado_reflexion_con_linea.png", reflected_with_line)

            print("Guardadas imágenes en output/")
            done = True

    elif event == cv2.EVENT_MOUSEMOVE:
        if len(points) == 1:
            display = actualizar_reflexion([x, y])
            cv2.imshow("Reflexion", display)

def actualizar_reflexion(temp_point=None):
    global clone, points

    if len(points) == 0:
        return clone.copy()
    elif len(points) == 1:
        p1 = np.array(points[0], dtype=np.float32)
        if temp_point is None:
            return clone.copy()
        p2 = np.array(temp_point, dtype=np.float32)
    else:
        p1 = np.array(points[0], dtype=np.float32)
        p2 = np.array(points[1], dtype=np.float32)

    h, w = clone.shape[:2]
    M_affine = get_affine_matrix(p1, p2)
    reflected = cv2.warpAffine(clone, M_affine, (w, h))

    display_reflected = reflected.copy()
    cv2.line(display_reflected, tuple(p1.astype(int)), tuple(p2.astype(int)), (255, 0, 0), 2)
    cv2.circle(display_reflected, tuple(p1.astype(int)), 5, (0, 0, 255), -1)
    cv2.circle(display_reflected, tuple(p2.astype(int)), 5, (0, 0, 255), -1)

    return display_reflected

def get_affine_matrix(p1, p2):
    dx, dy = p2 - p1
    ang = np.arctan2(dy, dx)

    T1 = np.array([[1, 0, -p1[0]],
                   [0, 1, -p1[1]],
                   [0, 0, 1]], dtype=np.float32)
    R = np.array([[ np.cos(-ang), -np.sin(-ang), 0],
                  [ np.sin(-ang),  np.cos(-ang), 0],
                  [0, 0, 1]], dtype=np.float32)
    Ref = np.array([[1, 0, 0],
                    [0, -1, 0],
                    [0, 0, 1]], dtype=np.float32)
    R_inv = np.linalg.inv(R)
    T2 = np.array([[1, 0, p1[0]],
                   [0, 1, p1[1]],
                   [0, 0, 1]], dtype=np.float32)
    M = T2 @ R_inv @ Ref @ R @ T1
    return M[:2, :]

def warp_reflection(img, p1, p2):
    h, w = img.shape[:2]
    M_affine = get_affine_matrix(np.array(p1, dtype=np.float32), np.array(p2, dtype=np.float32))
    return cv2.warpAffine(img, M_affine, (w, h))

def main():
    global clone
    ruta = "images/Gran_Canaria.jpg"
    clone = cv2.imread(ruta)
    if clone is None:
        raise FileNotFoundError("No se pudo cargar la imagen.")

    cv2.namedWindow("Original")
    cv2.namedWindow("Reflexion")

    cv2.imshow("Original", clone)

    cv2.setMouseCallback("Reflexion", mouse_callback)
    cv2.imshow("Reflexion", clone)

    print("Haz clic en el primer punto, mueve el ratón para definir la recta, y haz clic en el segundo punto.")
    cv2.waitKey(0)
    cv2.destroyAllWindows()


if __name__ == "__main__":
    main()


Haz clic en el primer punto, mueve el ratón para definir la recta, y haz clic en el segundo punto.
Guardadas imágenes en output/
